# Meta
In this notebook we will save out the brain, skull, and fMRI activations to then align in blender

# Setup

In [3]:
from notebook_viewer_functions import *
from functions import *
import numpy as np
import ants
import matplotlib.pyplot as plt
import os

proj_root = parent_directory()
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"
cache_folder = "/Volumes/GlyphA_R1/nvol_cache"
template_folder =  os.path.join(proj_root, "templates/")
output_folder = os.path.join(proj_root, "output/blender_alignment")

bold_image = ants.image_read(bold_stim_filepath)
t1_image = ants.image_read(t1_input_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_template = ants.image_read(mni_anat_filepath)
mni_mask = ants.image_read(mni_mask_filepath)

In [ ]:
def create_4D_volume_MVP(experimental_4D, control_3D_test_variant, output_folder="/Users/joachimpfefferkorn/repos/neurovolume/output/npy_seq"):
    fmri_volume = np.zeros(experimental_4D.shape)
    for time_index in range(experimental_4D.shape[3]):
        # Use the experimental_4D data for the current time slice
        current_slice = experimental_4D[:, :, :, time_index]
        
        # Calculate the density difference
        density = diff_with_thresh_3D(control_3D_test_variant, current_slice)
        
        # Update the fmri_volume with the density values
        fmri_volume[:, :, :, time_index] = density
        
        # Save the current time slice
        vol_frame = "{:03d}".format(time_index)
        np.save(f"{output_folder}/fmri_{vol_frame}.npy", fmri_volume[:, :, :, time_index])
        print(f"saved {vol_frame}")

In [ ]:
legacy_bold = create_4D_volume_MVP(bold_image.numpy(), bold_image.numpy()[:,:,:,0], output_folder=output_folder)

In [4]:
mask = generate_brain_mask(t1_image, mni_template, mni_mask)

Creating brain mask


In [5]:
mask_vol = create_normalized_volume(mask.numpy())

In [6]:
t1_vol = create_normalized_volume(t1_image.numpy())

In [7]:
fmri_vol = create_normalized_volume(bold_image.numpy())

In [11]:
print(fmri_vol.shape)

(64, 64, 35, 185)


The following names are important as they will be parsed

In [8]:
with open(f'{output_folder}/mask.npy', 'wb') as f:
    np.save(f, np.array(mask_vol))

In [9]:
with open(f'{output_folder}/anat.npy', 'wb') as f:
    np.save(f, np.array(t1_vol))

In [10]:
with open(f'{output_folder}/bold.npy', 'wb') as f:
    np.save(f, np.array(fmri_vol))